# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from src.simage import inference
import pandas as pd
import os
from pathlib import Path

# Load data

In [ ]:
root_dir = Path(os.getcwd()).parent

df = pd.read_excel(f"{root_dir}/data/immuno/data.xlsx", index_col=0)
feats = pd.read_excel(f"{root_dir}/data/immuno/feats.xlsx", index_col=0).index.values.tolist()
df = df.loc[:, feats + ['Age']]

inference(f"{root_dir}/data/immuno/model.ckpt", df)